In [31]:
pip install pandas scikit-learn transformers

Note: you may need to restart the kernel to use updated packages.


In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv('dark-patterns.csv')

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv('dark-patterns.csv')

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [34]:


# Encode labels for multi-class classification
import sklearn
import torch


label_encoder = sklearn.preprocessing.LabelEncoder()
train_labels = torch.tensor(label_encoder.fit_transform(train_data['Deceptive?']), dtype=torch.long)
test_labels = torch.tensor(label_encoder.transform(test_data['Deceptive?']), dtype=torch.long)

In [35]:
import transformers
num_classes = 3
model = transformers.RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_classes)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
##if you want to run code without GPU

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
import sklearn.preprocessing
import torch

# Load data
data = pd.read_csv('dark-patterns.csv')

# Drop rows with empty pattern strings
data = data.dropna(subset=['Pattern String'])

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenize and pad sequences
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train_inputs = tokenizer(list(train_data['Pattern String']), padding=True, truncation=True, return_tensors='pt')
test_inputs = tokenizer(list(test_data['Pattern String']), padding=True, truncation=True, return_tensors='pt')

# Encode labels for multi-class classification
import sklearn.preprocessing
label_encoder = sklearn.preprocessing.LabelEncoder()
train_labels = torch.tensor(label_encoder.fit_transform(train_data['Deceptive?']), dtype=torch.long)
test_labels = torch.tensor(label_encoder.transform(test_data['Deceptive?']), dtype=torch.long)

# Create DataLoader for training and testing
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Load pre-trained RoBERTa model for multi-class classification
num_classes = len(label_encoder.classes_)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_classes)

# Set up optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(3):
    model.train()  # Set the model to training mode
    for batch in train_dataloader:
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        logits = outputs.logits
        loss = criterion(logits, inputs['labels'])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Evaluate the model
model.eval()
# Implement evaluation code here using the test_dataloader


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\abhij\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [37]:
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in test_dataloader:
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)

        total_samples += inputs['labels'].size(0)
        total_correct += (predicted == inputs['labels']).sum().item()

accuracy = total_correct / total_samples
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 93.73%


In [39]:
import torch
import pickle

# Assuming 'model' is your trained PyTorch model
# Save the model to a pickle file
with open('model_final.pkl', 'wb') as f:
    torch.save(model.to('cpu'), f)


In [41]:
with open ('model_final.pkl', 'wb') as file:
  pickle.dump(model, file)

In [5]:
import torch
device = torch.device("cuda:0")
model = torch.load("classifier.pkl", map_location=device)

RuntimeError: Invalid magic number; corrupt file?

In [2]:
!pip install transformers

     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
     -------- ---------------------------- 30.7/129.4 kB 145.2 kB/s eta 0:00:01
     -------- ---------------------------- 30.7/129.4 kB 145.2 kB/s eta 0:00:01
     -------- ---------------------------- 30.7/129.4 kB 145.2 kB/s eta 0:00:01
     ----------- ------------------------- 41.0/129.4 kB 115.5 kB/s eta 0:00:01
     ----------------- ------------------- 61.4/129.4 kB 156.1 kB/s eta 0:00:01
     ----------------------- ------------- 81.9/129.4 kB 191.1 kB/s eta 0:00:01
     -------------------------- ---------- 92.2/129.4 kB 201.8 kB/s eta 0:00:01
     ---------------------------------- - 122.9/129.4 kB 232.7 kB/s eta 

In [7]:
import torch

# Save the model and tokenizer
torch.save(model_final.state_dict(), 'model_final.pth')
tokenizer.save_pretrained('tokenizer_directory')


NameError: name 'model_final' is not defined